In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt
import json
from datetime import datetime, timedelta

In [2]:
driver = webdriver.Chrome("C:/Users/김근영/chromedriver_win32.zip/chromedriver.exe")

C:\Users\김근영\AppData\Local\Temp\ipykernel_11816\3107180718.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/김근영/chromedriver_win32.zip/chromedriver.exe")


In [3]:
sm_genre_dict = { 
    3 : "가정/생활", #요리/살림
    4 : "가정/생활", #건강/취미
    5 : "경제/비즈니스", #경제/경영
#    6 : "문학", #고전
    7 : "자연/과학", #과학, 사회과
    8 : "강의지원도서", #대학교재/전문서적
#    9 : "라이트 노벨",
    10 : "문학", #소설/시/희곡
#    11 : "사전/기타",
    12 : "인문", #사회과학
#    13 : "수험서/자격증",
#    14 : "어린이",
    15 : "에세이/산문", # 에세이, 자기계발
#    16 : "여행",
#    17 : "역사",
    18 : "문화/예술", # 예술/대중문화
    19 : "외국어", #외국어
#    20 : "유아",
    21 : "인문", #인문학
#    22 : "인물/평전",
    23 : "에세이/산문",
#    24 : "잡지",
#    25 : "전집/중고전집",
#    26 : "종교/역학",
    27 : "가정/생활", #좋은부모
#    28 : "참고서/학습서",
#    29 : "청소년",
    30 : "컴퓨터/인터넷", # 컴퓨터/모바일
}

In [4]:

aladin_genre_dict = { 
    3 : "요리/살림",
    4 : "건강/취미",
    5 : "경제경영",
    # 6 :/ "고전",
    7 : "과학",
    8 : "대학교재/전문서적",
    # 9 : "라이트 노벨",
    10 : "소설/시/희곡",
    # 11 : "사전/기타",
    12 : "사회과학",
    # 13 : "수험서/자격증",
    # 14 : "어린이",
    15 : "에세이",
    # 16 : "여행",
    # 17 : "역사",
    18 : "예술/대중문화",
    19 : "외국어",
    # 20 : "유아",
    21 : "인문학",
    # 22 : "인물/평전",
    23 : "자기계발",
    # 24 : "잡지",
    # 25 : "전집/중고전집",
    # 26 : "종교/역학",
    27 : "좋은 부모",
    # 28 : "참고서/학습서",
    # 29 : "청소년",
    30 : "컴퓨터/모바일"
}

In [20]:
_index = "aladin_best_10"

In [21]:
# 칼럼 리스트 준비
book_list = []

In [22]:
present_date = str(datetime.utcnow() + timedelta(hours = 9))[:10]

In [23]:
driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=38409")

In [24]:
for sm_genre_num in range(27, 30):
    
    if(sm_genre_num == 3):
        sm_genre_id = 38409
        
    if(sm_genre_num == 6 or sm_genre_num == 9 or sm_genre_num == 11 or sm_genre_num == 13 or sm_genre_num == 14 or sm_genre_num == 16 or sm_genre_num == 17
       or sm_genre_num == 20 or sm_genre_num == 22 or sm_genre_num == 24 or sm_genre_num == 25 or sm_genre_num == 26 or sm_genre_num == 28
       or sm_genre_num == 29):
        sm_genre_num += 1
        continue

    elif(sm_genre_num == 4):
        sm_genre_id = 56388
    elif(sm_genre_num == 5):
        sm_genre_id = 38398
    elif(sm_genre_num == 6):
        sm_genre_id = 38414
    elif(sm_genre_num == 7):
        sm_genre_id = 38405
    elif(sm_genre_num == 8):
        sm_genre_id = 38422
   
    elif(sm_genre_num == 10):
        sm_genre_id = 38396

    elif(sm_genre_num == 12):
        sm_genre_id = 38404

    elif(sm_genre_num == 15):
        sm_genre_id = 56387

    elif(sm_genre_num == 18):
        sm_genre_id = 38402
    elif(sm_genre_num == 19):
        sm_genre_id = 38411

    elif(sm_genre_num == 21):
        sm_genre_id = 38403

    elif(sm_genre_num == 23):
        sm_genre_id = 38400

    elif(sm_genre_num == 27):
        sm_genre_id = 38413

        
    sm_genre_num += 1
    time.sleep(1)

    for i in range(1, 3):
        driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID="+str(sm_genre_id)+"&page="+str(i)+"&cnt=300&SortOrder=1")
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 1위 책 클릭(대여 횟수 긁어오기 위함)

        books = soup.select('div.ss_book_box')
        def get_star(book):
            stars = book.select_one("div.ss_book_list img")
            if stars is None:
                return 0
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s10.gif":
                return 5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s9.gif":
                return 4.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s8.gif":
                return 4
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s7.gif":
                return 3.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s6.gif":
                return 3
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s5.gif":
                return 2.5
            elif stars['src'] == "//image.aladin.co.kr/img/common/star_s4.gif":
                return 2
            else:
                return 0
        
        for book in books:
            try: #일반적인 경우
                title = book.select('a.bo3')[0].text
                rank = book.select("td")[0].text
                
                li_tags = book.find_all('li')
                second_li_tag = li_tags[2] # 2. 두 번째 li 태그 선택
                info = second_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                author=info.split('|')[0]
                publisher=info.split('|')[1]
                date=info.split('|')[2] 
                
                price = book.select("span.ss_p2 > b > span")[0].text
                sm_genre = sm_genre_dict[sm_genre_num - 1]
                aladin_genre = aladin_genre_dict[sm_genre_num - 1]
                
                star = get_star(book)
                
                i += 1
                

            except IndexError: #행사 상품
                title = book.select('a.bo3')[0].text
                price = book.select("span.ss_p2 > b > span")[0].text
                rank = book.select("td")[0].text
                
                star = get_star(book)
                try: #only 행사 상품
                    li_tags = book.find_all('li')
                    third_li_tag = li_tags[3] # 2. 두 번째 li 태그 선택
                    info = third_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                    author=info.split('|')[0]
                    publisher=info.split('|')[1]
                    date=info.split('|')[2]
                    sm_genre = sm_genre_dict[sm_genre_num - 1]
                    aladin_genre = aladin_genre_dict[sm_genre_num - 1]
                    

                except IndexError: #행사 상품은 아닌데, 출판사가 없음.
                    second_li_tag = li_tags[2]
                    info = second_li_tag.get_text()
                    author=info.split('|')[0]
                    publisher=None
                    date=info.split('|')[1] 
                    sm_genre = sm_genre_dict[sm_genre_num - 1]
                    aladin_genre = aladin_genre_dict[sm_genre_num - 1]

               
            book_list.append([rank, title, author, publisher, date, price, star, aladin_genre, sm_genre, present_date])
            i += 1

In [25]:
for i in range(1, 3):
    driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=EBookBestseller&BranchType=9&CID=38401&page="+str(i)+"&cnt=300&SortOrder=1")
    time.sleep(0.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 1위 책 클릭(대여 횟수 긁어오기 위함)
    books = soup.select('div.ss_book_box')
    def get_star(book):
        stars = book.select_one("div.ss_book_list img")
        if stars is None:
            return 0
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s10.gif":
            return 5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s9.gif":
            return 4.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s8.gif":
            return 4
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s7.gif":
            return 3.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s6.gif":
            return 3
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s5.gif":
            return 2.5
        elif stars['src'] == "//image.aladin.co.kr/img/common/star_s4.gif":
            return 2
        else:
            return 0
    for book in books:
        title = book.select('a.bo3')[0].text
        price = book.select("span.ss_p2 > b > span")[0].text
        rank = book.select("td")[0].text
                
        star = get_star(book)
        try: #일반적인 경우
            
            li_tags = book.find_all('li')
            second_li_tag = li_tags[2] # 2. 두 번째 li 태그 선택
            info = second_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

            author=info.split('|')[0]
            publisher=info.split('|')[1]
            date=info.split('|')[2] 
            price = book.select("span.ss_p2 > b > span")[0].text
            sm_genre = sm_genre_dict[sm_genre_num]            
                
            i += 1
            

        except IndexError: #행사 상품

            try: #only 행사 상품
                li_tags = book.find_all('li')
                third_li_tag = li_tags[3] # 2. 두 번째 li 태그 선택
                info = third_li_tag.get_text() # 3. 두 번째 li 태그 내부의 첫 번째 내용 가져오기

                author=info.split('|')[0]
                publisher=info.split('|')[1]
                date=info.split('|')[2]
                sm_genre = sm_genre_dict[sm_genre_num]
                aladin_genre = aladin_genre_dict[sm_genre_num]
            
            except IndexError: #행사 상품은 아닌데, 출판사가 없음.
                second_li_tag = li_tags[2]
                info = second_li_tag.get_text()
                author=info.split('|')[0]
                publisher=None
                date=info.split('|')[1] 
                sm_genre = sm_genre_dict[sm_genre_num]
                aladin_genre = aladin_genre_dict[sm_genre_num]
            

        book_list.append([rank, title, author, publisher, date, price, star, aladin_genre, sm_genre, present_date])
        i += 1

In [26]:
#func = lambda x: name_mapping.get(x, x) # name_mapping 딕셔러니 값으로 장르 설정

df = pd.DataFrame(book_list, columns = ["aladin_rank", "subject", "aladin_writer", "aladin_publisher", "aladin_date","aladin_price", "aladin_review", "aladin_genre", "sm_genre", "present_date"])

df['present_date'] = present_date #time_stamp 속성 생성



In [12]:
df

,aladin_rank,subject,aladin_writer,aladin_publisher,aladin_date,aladin_price,aladin_review,aladin_genre,sm_genre,present_date
0,\n\n1. \n,세상에서 가장 쉬운 본질육아,지나영 (지은이),21세기북스,2022년 9월,"15,040",5.0,좋은 부모,가정/생활,2023-02-24
1,\n\n2. \n,초등입학 엄마의 비밀노트,지서영 (지은이),이페이지,2023년 1월,"1,000",5.0,좋은 부모,가정/생활,2023-02-24
2,\n\n3. \n,자발적 방관육아,최은아 (지은이),쌤앤파커스,2023년 1월,"12,000",5.0,좋은 부모,가정/생활,2023-02-24
3,\n\n4. \n,"어머니, 사교육을 줄이셔야 합니다",정승익 (지은이),메이트북스,2023년 1월,"10,200",5.0,좋은 부모,가정/생활,2023-02-24
4,\n\n5. \n,어떻게 말해줘야 할까,"오은영 (지은이), 차상미 (그림)",김영사,2020년 11월,"11,070",5.0,좋은 부모,가정/생활,2023-02-24
...,...,...,...,...,...,...,...,...,...,...
195,\n\n96. \n,Do it! Vue.js 입문,장기효 (지은이),이지스퍼블리싱,2018년 2월,"10,500",4.5,좋은 부모,컴퓨터/인터넷,2023-02-24
196,\n\n97. \n,스프링 부트 핵심 가이드,장정우 (지은이),위키북스,2022년 8월,"20,000",5.0,좋은 부모,컴퓨터/인터넷,2023-02-24
197,\n\n98. \n,Do it! 조코딩의 프로그래밍 입문,"김형태, 조동근(조코딩) (지은이)",이지스퍼블리싱 (주),2022년 7월,"12,000",5.0,좋은 부모,컴퓨터/인터넷,2023-02-24
198,\n\n99. \n,리액트 네이티브를 다루는 기술,김민준 (지은이),길벗,2021년 11월,"30,100",5.0,좋은 부모,컴퓨터/인터넷,2023-02-24


In [27]:

# 데이터 전처리

df['aladin_rank'] = df["aladin_rank"].str.replace(pat = r'[^A-Za-z0-9가-힣]', repl = r' ', regex = True) #공백(엔터)

df['aladin_price'] = df['aladin_price'].str.replace(pat=',', repl = '',regex=True) #, 삭제
df['aladin_price'] = pd.to_numeric(df['aladin_price'])


df['aladin_date'] = df['aladin_date'].str.replace(pat='년 ', repl = '-',regex=True)#년 -> -
df['aladin_date'] = df['aladin_date'].str.replace(pat='월', repl = '',regex=True) #월 -> 

df['aladin_date'] = df['aladin_date'].str.strip()
df['aladin_date'] = pd.to_datetime(df['aladin_date'])

In [28]:
df2 = df

In [29]:
from elasticsearch import Elasticsearch, helpers
import configparser

In [30]:
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [31]:
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

C:\Users\김근영\AppData\Local\Temp\ipykernel_11816\2962435662.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [32]:
if es.indices.exists(index = _index + present_date):
    pass
else:
    resp = es.indices.create(index = _index + present_date, body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori": {
                        "tokenizer": "nori_tokenizer"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "subject": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_genre": {
                    "type": "keyword"
                },
                "aladin_writer": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_publisher": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "aladin_date": {
                    "type": "text",
                    "fields" : {
                        "keyword" : {
                            "type" : "keyword"
                        }
                    }
                },
                "sm_genre": {
                    "type": "keyword"
                },
                "aladin_price": {
                    "type": "integer"
                },
                "aladin_review": {
                    "type": "float"
                },
                "present_date": {
                    "type": "date"
                }
            }
        }
    })

C:\Users\김근영\AppData\Local\Temp\ipykernel_11816\4125453572.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.indices.create(index = _index + present_date, body = {


In [33]:
for k in range(len(df2)) :
    es.index(index= _index + present_date,
         document = {
         "aladin_rank" : str(df2.loc[k].aladin_rank),
         "subject" : str(df2.loc[k].subject),
         "aladin_writer": str(df2.loc[k].aladin_writer),
         "aladin_publisher": str(df2.loc[k].aladin_publisher),
         "aladin_date": str(df2.loc[k].aladin_date),
         "aladin_price": str(df2.loc[k].aladin_price),
         "aladin_review": str(df2.loc[k].aladin_review),
         "aladin_genre": str(df2.loc[k].aladin_genre),
         "sm_genre": str(df2.loc[k].sm_genre),
         "present_date": present_date
 })